In [87]:
#global
INPUT_LOCATION = './inputs/day7.txt'
SEPARATOR = '\n'

In [88]:
clean_data = list(map(lambda x: x.rstrip() if len(x) > 1 else x,open(INPUT_LOCATION, 'r').readlines()))  

In [89]:
# nodes for my bag graph
class Bag:
    
    def __init__(self, bag_color, content={}):
        self.color = bag_color
        self.content = content
        self.discovered = False
    
    def __getitem__(self, key):
        return self.content[key]

    def __eq__(self, other):
        if isinstance(other, Bag):
            return self.color == other.color
        return False
    
    def __lt__(self, other):
        return self.color < other.color
    
    def __repr__(self):
        return 'a {} bag {}'.format(self.color,self.content) 
    
    def __str__(self):
        return 'a {} bag {}'.format(self.color,self.content) 

    def __len__(self):
        return len(self.content)

In [90]:
# graph representing bags dependencies
# built from scratch... just for fun..
class BagsGraph:
    def __init__(self, bags = []):
        self.bags = bags
        self.dep = []

    def __repr__(self):
        return 'graph with {} Nodes'.format(len(self.bags))
    
    def __str__(self):
        return 'graph with {} Nodes'.format(len(self.bags))
    
    def __getitem__(self, key):
        return self.findBag(key)
    
    def reset(self):
        for node in self.bags:
            node.discovered = False
    
    def findIndex(self, bag):
        return self.bags.index(bag)
    
    def findBag(self, bag_color):
        return next(x for x in self.bags if x.color == bag_color)
    
    def get_parents(self, bag):
        return [x for x in self.bags if bag.color in x.content]
    
    def markVisited(self,bag):
        self.bags[self.findIndex(bag)].discovered = True
    
    def __recursive_parent_finder(self,bag, container = [],prefix = ''):
        self.markVisited(bag)
        # print(prefix,'*',bag, 'has :')
        p = self.get_parents(bag)
        if len(p) > 0 :
            for x in p:
                if x.discovered == False:
                    container.append(x)
                    #print ('{}-{}'.format(prefix,x))
                    self.__recursive_parent_finder(x,container,prefix+'\t')
                # else: print("ALREADY VISITED!")
        # else: print('{}-no parent!'.format(prefix))
        container.sort()
        return len(container)
    
    def count_parents(self, bag):
        self.reset()
        result = []
        return self.__recursive_parent_finder(bag,result)
        
    def dfs(self, bag):
        result = []
        self.reset()
        return self.__dfs(bag, result)
        
    def __dfs(self, bag, result = []):
        i = self.findIndex(bag)
        #print(result)
        self.bags[i].discovered = True
        result.append(self.bags[i])
        for el in self.bags[i].content:
            child = self.findBag(el)
#             print(child)
            if not child.discovered:
#                 print('discovered: ', child.color)
                self.__dfs(child, result)
        return result

    def append(self, Bag):
        self.bags.append(Bag)

In [92]:
# cleaning data and building a bag dependecy graph
ignored = ['bags,','bags.','bags','bag','bag,','contain']

rules = []
bg = BagsGraph([])
clean = dict()

for raw_rule in clean_data:
    rule = raw_rule.split(' ')
    to_be_deleted = []
    for el in rule:
        if el in ignored:
            to_be_deleted.append(el)
    for el in to_be_deleted:
        rule.remove(el)
    
   # print(rule)
    key = rule[0] +' '+ rule[1]
    clean[key] = {}
    for i in range(3,len(rule)):
        if i % 3 == 0 and i + 1 < len(rule):
            clean[key].update({rule[i] +' '+ rule[i+1]: int(rule[i-1])} )
    b = Bag(key, clean[key])
    bg.append(b)

print(bags_graph)

graph with 594 Nodes


In [93]:
bg = bags_graph

In [94]:
# first question 
bags_graph.count_parents(Bag('shiny gold'))

246

In [100]:
# second question 
# weighted graph travelsal
def sumbag(bag, p='', q=1):
    sum=0
    if bag.content == {}:
        print(p,q,bag.color, 'contains nothing. ' )
        return q
    else:
        print(p,q,bag.color, 'contains: ')
        for color in bag.content:
            child = bg[color]
            sum += sumbag(child,p+'\t',bag[color]) 
        return sum*q+q
    
sumbag(bg['shiny gold'])-1


 1 shiny gold contains: 
	 5 dark salmon contains: 
		 3 pale turquoise contains: 
			 3 vibrant red contains nothing. 
		 5 faded tan contains: 
			 1 muted plum contains: 
				 4 dull red contains: 
					 2 dotted maroon contains nothing. 
					 1 posh salmon contains nothing. 
					 3 dotted chartreuse contains nothing. 
					 2 dim yellow contains: 
						 1 dotted chartreuse contains nothing. 
				 1 dotted maroon contains nothing. 
				 1 vibrant red contains nothing. 
				 4 bright chartreuse contains: 
					 1 muted gold contains nothing. 
					 1 dotted maroon contains nothing. 
			 1 posh salmon contains nothing. 
		 1 mirrored chartreuse contains nothing. 
	 2 wavy purple contains: 
		 3 dim yellow contains: 
			 1 dotted chartreuse contains nothing. 
		 1 posh salmon contains nothing. 
	 5 dark cyan contains: 
		 3 dotted maroon contains nothing. 
		 2 vibrant red contains nothing. 
	 5 dull chartreuse contains: 
		 3 mirrored black contains nothing. 
		 3 dotted salmon con

2976